In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize as op
import scipy.signal as sig
from   scipy import stats
from   scipy import fftpack
import astropy
from   astropy.io import fits as pyfits
import random

import csv
import sys
import os
import importlib as imp
import glob
from   timeit import default_timer as timer
import warnings
import progressbar
import argparse
import json

import lightkurve as lk
import exoplanet as exo
import theano.tensor as T
import pymc3 as pm
import corner

from archeopteryx.constants import *
from archeopteryx.utils import *
from archeopteryx.Planet import *
from archeopteryx.LiteCurve import *
import archeopteryx.io as io
import archeopteryx.detrend as detrend
import archeopteryx.noise as noise


# flush buffer to avoid mixed outputs from progressbar
sys.stdout.flush()

# turn off FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

# start program timer
global_start_time = timer()

In [2]:
# select mission, target, and paths

MISSION = "Kepler"
TARGET  = "K00137"
PRIMARY_DIR = '/Users/research/projects/archeopteryx/'
TRACE_FILE  = '/Users/research/projects/archeopteryx/Traces/K00137/K00137_indep_ttvs_200710a.fits'

# Make sure the necessary paths exist

In [3]:
# directory in which to find lightcurve data
if MISSION == 'Kepler': DOWNLOAD_DIR = PRIMARY_DIR + 'MAST_downloads/'
if MISSION == 'Simulated': DOWNLOAD_DIR = PRIMARY_DIR + 'Simulations/'

# directories in which to place pipeline outputs    
FIGURE_DIR    = PRIMARY_DIR + 'Figures/' + TARGET + '/'
TRACE_DIR     = PRIMARY_DIR + 'Traces/' + TARGET + '/'
QUICK_TTV_DIR = PRIMARY_DIR + 'QuickTTVs/' + TARGET + '/'
DLC_DIR       = PRIMARY_DIR + 'Detrended_lightcurves/' + TARGET + '/'
NOISE_DIR     = PRIMARY_DIR + 'Noise_models/' + TARGET + '/'

# check if all the paths exist and create them if not
if os.path.exists(FIGURE_DIR) == False:
    os.mkdir(FIGURE_DIR)
    
if os.path.exists(TRACE_DIR) == False:
    os.mkdir(TRACE_DIR)
    
if os.path.exists(QUICK_TTV_DIR) == False:
    os.mkdir(QUICK_TTV_DIR)
    
if os.path.exists(DLC_DIR) == False:
    os.mkdir(DLC_DIR)
    
if os.path.exists(NOISE_DIR) == False:
    os.mkdir(NOISE_DIR)

# Get HBM TTV model posteriors

In [10]:
hbm_trace = pyfits.open(TRACE_FILE)
header = hbm_trace[0].header
hdulist = pyfits.HDUList(hbm_trace)

OFFSET_1 = hbm_trace['OFFSET_1'].data
OFFSET_2 = hbm_trace['OFFSET_2'].data

TTS_1 = hbm_trace['TTS_1'].data
TTS_2 = hbm_trace['TTS_2'].data

hbm_trace.close()

# These are the transit times for K-18c (P = 7 days)

In [15]:
ntrans = TTS_1.shape[1]

for i in range(ntrans):
    print(np.median(TTS_1[:,i]), '\t', np.std(TTS_1[:,i]))

127.769325171819 	 0.001058216335278474
135.4091462526531 	 0.0005236276801888335
150.69373898026362 	 0.0005281798604430748
158.33779658302225 	 0.000664963258469857
173.6116419069059 	 0.0006377164167516063
181.2539591292844 	 0.0006360130780032567
188.89758222649132 	 0.000553231481640366
204.180082543949 	 0.0010448832768203914
211.8240875554987 	 0.000549702328092986
219.45966876711407 	 0.0017215632068779303
227.10262047937164 	 0.0017024663034423564
234.74924737776337 	 0.0009485371828393592
242.38859349500763 	 0.0014486003650862978
250.02730859578554 	 0.001788252299722849
257.667416736789 	 0.0011125703234131025
265.314214799366 	 0.0006974535366019617
272.9570800297179 	 0.0008421961319589045
280.59956172026637 	 0.0011107437194995083
288.241301536149 	 0.0006519481101522831
295.8838869427201 	 0.0006858023181123587
303.52442096429104 	 0.0012355627273337585
311.1677679785532 	 0.000815115603509332
318.80900298481083 	 0.0007575653955484254
326.45025438042126 	 0.00108106566

# These are the transit times for K-18d (P = 14 days)

In [16]:
ntrans = TTS_2.shape[1]

for i in range(ntrans):
    print(np.median(TTS_2[:,i]), '\t', np.std(TTS_2[:,i]))

128.1513119599484 	 0.0006025157231633182
157.873959612127 	 0.0012941250426428295
172.7322005327333 	 0.0006858655753583409
187.59322463489286 	 0.0007137994768382554
202.44942009525812 	 0.0005838395814452169
217.30956282050064 	 0.0006080771462774008
232.16880527945506 	 0.001237525076953817
247.02751865543266 	 0.001113871846786686
261.885064741037 	 0.0007080796624686924
276.7434621872353 	 0.0006411264021138765
291.5999763549652 	 0.0007524949245011562
306.45818444396195 	 0.0006508054617898964
321.31626240066385 	 0.0006629051561895721
336.17617665801015 	 0.0006236688583605913
365.89425137826856 	 0.0007331181205974686
380.7522031682676 	 0.0007481678185111483
395.6131539647878 	 0.00084191436014369
425.3313427731743 	 0.0007316830962519811
440.19131818599743 	 0.0006552484074363782
455.0523531658225 	 0.0006618266356229875
484.77117674257386 	 0.0006958235867989963
499.62958003282665 	 0.0006479462476747555
514.4876834463547 	 0.0007324277927848003
529.3452959601784 	 0.000741